In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import  LabelBinarizer

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

#### 위 코드가 안될 시 아래 방식으로 mnist csv 가져옴

In [2]:
df_train = pd.read_csv('./mnist_csv/mnist_train.csv', header=None)
df_test = pd.read_csv('./mnist_csv/mnist_test.csv', header=None)

train_list = list(df_train)
test_list = list(df_test)

train_images = df_train.iloc[:,1:].values
test_images = df_test.iloc[:,1:].values

lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
train_labels = lb1.fit_transform(df_train.iloc[:,0].values)
test_labels = lb2.fit_transform(df_test.iloc[:,0].values)

mnist = {'train':{
        'images':train_images,
        'labels':train_labels},
        'test':{
        'images':test_images,
        'labels':test_labels}
         }

### softmax classifier 

In [10]:
# parameter
learning_rate = 0.01
training_epochs = 15
batch_size =100
display_step = 1

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, 10])

w = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# construct model
activation = tf.nn.softmax(tf.matmul(x,w)+b)

# minimize cost
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(activation), reduction_indices=1)) # cross entropy
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) # gradient descent

init = tf.global_variables_initializer()

# launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # training cycle <- mnist database 있을 때,
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # fit training using batch data
            sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys})
            # compute average loss
            avg_cost += sess.run(cost, feed_dict={x:batch_xs, y:batch_ys})/total_batch
            
        if epoch % display_step == 0:
            print('Epoch:', '%04d'%(epoch+1), 'cost=','{:.9f}'.format(avg_cost))
            correc_prediction = tf.equal(tf.arg_max(activation,1), tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correc_prediction, tf.float32))
            print('Accuracy: ',sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels}))
            
    print('Optimization Finished!')

            
        
#     # training cycle <- mnist 없을 때
#     feed_train = {x:mnist['train']['images'], y:mnist['train']['labels']}
#     feed_test = {x:mnist['test']['images'], y:mnist['test']['labels']}
#     for step in range(501):
#         sess.run(optimizer, feed_dict=feed_train)
        
#         if step%20==0:
#             print(step, sess.run(cost, feed_dict=feed_train))
#         if step%100==0:
#             correc_prediction = tf.equal(tf.arg_max(activation,1), tf.argmax(y,1))
#             accuracy = tf.reduce_mean(tf.cast(correc_prediction, tf.float32))
#             print(sess.run(accuracy, feed_dict=feed_test))

Epoch: 0001 cost= 1.177053162
Accuracy:  0.8561
Epoch: 0002 cost= 0.662652940
Accuracy:  0.8744
Epoch: 0003 cost= 0.550684364
Accuracy:  0.8822
Epoch: 0004 cost= 0.496750795
Accuracy:  0.8879
Epoch: 0005 cost= 0.463723682
Accuracy:  0.8905
Epoch: 0006 cost= 0.440943577
Accuracy:  0.8952
Epoch: 0007 cost= 0.423945613
Accuracy:  0.8979
Epoch: 0008 cost= 0.410667528
Accuracy:  0.8999
Epoch: 0009 cost= 0.399911019
Accuracy:  0.9002
Epoch: 0010 cost= 0.390939410
Accuracy:  0.9031
Epoch: 0011 cost= 0.383391962
Accuracy:  0.9048
Epoch: 0012 cost= 0.376766298
Accuracy:  0.9057
Epoch: 0013 cost= 0.370985510
Accuracy:  0.9067
Epoch: 0014 cost= 0.365930335
Accuracy:  0.908
Epoch: 0015 cost= 0.361394126
Accuracy:  0.9084
Optimization Finished!


### NN with initialization & dropout

#### Xavier initialization

In [3]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs+n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0/ (n_inputs+n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [4]:
# parameter
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# tf graph input
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, 10])

# 기본 초기값
# w1 = tf.Variable(tf.random_normal([784,256]))
# w2 = tf.Variable(tf.random_normal([256,256]))
# w3 = tf.Variable(tf.random_normal([256,10]))

# Xavier initialization
w1 = tf.get_variable('w1', shape=[784, 256], initializer=xavier_init(784, 256))
w2 = tf.get_variable('w2', shape=[256, 256], initializer=xavier_init(256, 256))
w3 = tf.get_variable('w3', shape=[256, 128], initializer=xavier_init(256, 128))
w4 = tf.get_variable('w4', shape=[128, 64], initializer=xavier_init(128, 64))
w5 = tf.get_variable('w5', shape=[64, 10], initializer=xavier_init(64, 10))
b1 = tf.Variable(tf.random_normal([256]))
b2 = tf.Variable(tf.random_normal([256]))
b3 = tf.Variable(tf.random_normal([128]))
b4 = tf.Variable(tf.random_normal([64]))
b5 = tf.Variable(tf.random_normal([10]))

# 기본 모델
# l1 = tf.nn.relu(tf.matmul(x,w1)+b1)
# l2 = tf.nn.relu(tf.matmul(l1,w2)+b2) # hidden layer with relu activation
# hypothesis = tf.matmul(l2,w3)+b3 # no need to use softmax here

# more deep & Dropout
dropout_rate = tf.placeholder('float')
_l1 = tf.nn.relu(tf.add(tf.matmul(x, w1), b1))
l1 = tf.nn.dropout(_l1, dropout_rate)
_l2 = tf.nn.relu(tf.add(tf.matmul(l1, w2), b2))
l2 = tf.nn.dropout(_l2, dropout_rate)
_l3 = tf.nn.relu(tf.add(tf.matmul(l2, w3), b3))
l3 = tf.nn.dropout(_l3, dropout_rate)
_l4 = tf.nn.relu(tf.add(tf.matmul(l3, w4), b4))
l4 = tf.nn.dropout(_l4, dropout_rate)

hypothesis = tf.add(tf.matmul(l4, w5), b5)

# define cost & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, y))  # softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initializing the variables
init = tf.global_variables_initializer()

# launch the graph
with tf.Session() as sess:
    sess.run(init)

    # training cycle <- mnist database 있을 때,
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)

        # loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, dropout_rate: 0.7})
            # compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, dropout_rate: 0.7}) / total_batch

        if epoch % display_step == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost=', '{:.9f}'.format(avg_cost))
            correc_prediction = tf.equal(tf.arg_max(hypothesis, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correc_prediction, tf.float32))
            print('Accuracy: ',
                  sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, dropout_rate: 1.0}))

    print('Optimization Finished!')

            
        
#     # training cycle <- mnist 없을 때
#     feed_train = {x:mnist['train']['images'], y:mnist['train']['labels']}
#     feed_test = {x:mnist['test']['images'], y:mnist['test']['labels']}
#     for step in range(501):
#         sess.run(optimizer, feed_dict=feed_train)
        
#         if step%20==0:
#             print(step, sess.run(cost, feed_dict=feed_train))
#         if step%100==0:
#             correc_prediction = tf.equal(tf.arg_max(activation,1), tf.argmax(y,1))
#             accuracy = tf.reduce_mean(tf.cast(correc_prediction, tf.float32))
#             print(sess.run(accuracy, feed_dict=feed_test))

Epoch: 0001 cost= 0.712773009
Accuracy:  0.9327
Epoch: 0002 cost= 0.253829555
Accuracy:  0.9605
Epoch: 0003 cost= 0.190043738
Accuracy:  0.9648
Epoch: 0004 cost= 0.157804752
Accuracy:  0.9671
Epoch: 0005 cost= 0.128981147
Accuracy:  0.9721
Epoch: 0006 cost= 0.117702882
Accuracy:  0.9745
Epoch: 0007 cost= 0.110079603
Accuracy:  0.9755
Epoch: 0008 cost= 0.096193713
Accuracy:  0.9765
Epoch: 0009 cost= 0.091750590
Accuracy:  0.9765
Epoch: 0010 cost= 0.085452255
Accuracy:  0.9767
Epoch: 0011 cost= 0.080557575
Accuracy:  0.9789
Epoch: 0012 cost= 0.076146642
Accuracy:  0.9778
Epoch: 0013 cost= 0.072642161
Accuracy:  0.9789
Epoch: 0014 cost= 0.067537851
Accuracy:  0.9796
Epoch: 0015 cost= 0.066766725
Accuracy:  0.9805
Optimization Finished!


### 정리

- Softmax vs Neural Nets for MNIST => 91.4% vs 94.4%
- Xavier initialization: 97.8%
- Deep Neural Nets and Dropout: 98%
- Adam optimizer